In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [ ]:
# create a Dataframe from draws from random normal dist, specifying index and cols
df = DataFrame(np.random.randn(10, 5),
               index=[i for i in 'IN OH MI KY IL CO WY NM CA OR'.split(' ')],
               columns=['feat_' + str(i) for i in range(1, 6)])

In [ ]:
more_data = {'state': [i for i in 'IN OH MI KY IL CO WY NM CA OR'.split(' ')],
             'is_big': [False, True, True, False, np.nan, True, False, True, True, True],
             'nickname': ['Hoosier', 'Buckeye', 'Wolverine', 'Bluegrass', 'Prairie',
                          'Centennial', 'Equality', 'Enchantment', 'Golden', 'Beaver']}

In [ ]:
# notice we're not setting an index value here.
df2 = DataFrame(more_data)

In [ ]:
"""as an aside, note that we can use the from_dict() method here as well.
this method takes an optional argument, orient'.  If keys should be columns
pass 'columns' to orient (default behavior).  If keys should be row indices 
pass 'index' to this parameter.  See below...
"""

df2 = DataFrame.from_dict(more_data, orient='index')
print(df2)

df2 = DataFrame.from_dict(more_data, orient='columns')
print(df2)

In [ ]:
# Let's join these DataFrames
data = df.merge(df2, how='left', left_index=True, right_on='state')

# Notice the resulting indices
print(data.index)

In [ ]:
# Let's merge them again, this time we'll specify indices for df2.
# This will introduce some redundancy into our DataFrame
# df2.set_index('state', inplace=True) is a better alternative.
df2 = DataFrame(more_data, index=more_data['state'])
df2.drop(columns='state', inplace=True)

# Executing the merge again.
data = df.merge(df2, how='left', left_index=True, right_index=True)

In [ ]:
# Now notice the resulting indices.
print(data.index)

In [ ]:
# Perhaps we want the state code as a feature and not the index
data.reset_index(drop=False, inplace=True)

In [ ]:
# notice the name of the state code feature
print(data.columns)

In [ ]:
print(data)

In [ ]:
# renaming cols inplace
data.rename(columns={'index': 'state'}, inplace=True)

In [ ]:
# setting index back to state code
data.set_index('state', inplace=True)

In [ ]:
# notice that the index now has a name attribute
print(data.index)

In [ ]:
# Let's add another feature.
data['year'] = '1997'

In [ ]:
# Let's make another df to concat our first dataframe
another_df = DataFrame(np.random.randn(10, 5),
                       index=[i for i in 'IN OH MI KY IL CO WY NM CA OR'.split(' ')],
                       columns=['feat_' + str(i) for i in range(1, 6)])

In [ ]:
another_df = another_df.merge(df2, how='left', left_index=True, right_index=True)

another_df['year'] = '2001'

# and we'll add another column to this df.
another_df['quarter'] = 'q1'

In [ ]:
# concatenate the two dataframes
# axis arg is 0 by default
final_df = pd.concat([data, another_df], axis=0)

In [ ]:
print(final_df)

In [ ]:
# fillna for quarter...this wasn't a feature in big_df.
final_df['quarter'].fillna(value='q1', inplace=True)

In [ ]:
# now we're going to double the length of our primary DataFrame.
# Be cautious with aliasing...
copied = final_df

copied['quarter'] = 'q2' # chaning the quarter column value

print(final_df['quarter']) # insepect the original column of the df from which we copied

In [ ]:
# That wasn't good.  Let's change the 'quarter' vale of final_df back to 'q1'
final_df['quarter'] = 'q1'

In [ ]:
# Use the .copy() instead. Copy takes one optional argument, deep.
# The param is True by default. deep=False returns a shallow copy.
copied = final_df.copy()
copied['quarter'] = 'q2'

In [ ]:
# confirming that the change to copied
# had no impact on final_df
print(final_df['quarter'])

In [ ]:
# iterating over the feature columns and modifying them randomly
for i in ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5']:
    copied[i] = np.vectorize(lambda arg: np.random.randn())(copied[i])

In [ ]:
final_df = pd.concat([final_df, copied], axis=0)

In [ ]:
# Let's pivot this and join an aggregate field to our final_df.
# Say, for some reason, we need the max across annual quarters of the feature values by year
pivoted = final_df.pivot_table(index=final_df.index,
                               columns='year',
                               values=['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5'],
                               aggfunc=np.max)

In [ ]:
# we now have MultiIndexed columns where column indices are represented as tuples
print(pivoted.columns)
print(pivoted.index)
print(pivoted)

In [ ]:
# MultiIndexed rows and columns are totally fine, but what if we want to flatten them out...
pivoted.columns = ['_'.join(col) for col in pivoted.columns]

In [ ]:
print(pivoted)

In [ ]:
# Maybe it makes more sense to pivot this way, if we want to join these data.
pivoted = final_df.pivot_table(index=[final_df.index, 'year'],
                               values=['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5'],
                               aggfunc=np.max)

In [ ]:
# now we have MultiIndexed rows.
print(pivoted.columns)
print(pivoted.index)
print(pivoted)

In [ ]:
# Quick aside:
# we could aggregate over components of the multi-index.
# note that the level keyword in Series and DataFrame aggregationis deprecated. Use groupby instead.
# pivoted.sum(level='year')

pivoted.groupby(level=1).sum()

In [ ]:
# note that our pivoted data contains the same feature names as were in our primary data
# If we merge our pivoted data to our original dataset Pandas will handle this by
# appending an _x and _y to these feature names. We'll change these names first.
pivoted.columns = [i + '_ann_max' for i in pivoted.columns]

In [ ]:
# merging pivoted data to our final_df
final_df = final_df.merge(pivoted,
                          how='left',
                          left_on=[final_df.index, 'year'],
                          right_index=True)

In [ ]:
# sorting index
final_df.sort_index(ascending=True,
                    inplace=True)

In [ ]:
print(final_df.index)

In [ ]:
# sorting on additional cols
final_df.sort_values(['year', 'quarter'],
                     ascending=[False, True],
                     inplace=True)

In [ ]:
print(final_df[['year', 'quarter']])

In [ ]:
# maybe we want to drop records with a certain nickname
# we can send the indices of the data with certain properties to a list
location = final_df.index[(final_df['nickname'].isin(['Buckeye', 'Bluegrass']))].tolist()

#final_df.drop(index=location, inplace=True)
# we'll hold off on dropping them for now though.

In [ ]:
# drop rows with missing values

#final_df.dropna(axis=0, inplace=True)

# let's hold off on this as well.  Instead we'll impute.

# first let's inspect which columns have missing data
final_df.isnull().any()

In [ ]:
# we can use this bool array to subset our actual column names.
# this is much easier to see columns with missing data.
missing = final_df.columns[final_df.isnull().any()].tolist()
print(missing)

In [ ]:
for col in  missing:
    final_df[col].fillna(value='maybe', inplace=True)

In [ ]:
for i in final_df['is_big'].unique():
    final_df[i] = np.vectorize(lambda x: 1 if x == True else 0)(final_df['is_big'])

In [ ]:
# Let's take a look at this DataFrame
print(final_df.head())

In [ ]:
# let's do some groupby operations.
# This just establishes a groupby object
gb1 = final_df.groupby(['is_big'])

In [ ]:
# we can call aggregation functions on this object
gb1.mean()

In [ ]:
gb1[['feat_1', 'feat_2']].mean()

In [ ]:
gb1[['feat_1', 'feat_2']].min()

In [ ]:
# confirming the feat_1 min for is_big = False
final_df.loc[final_df['is_big'] == False, 'feat_1'].min()

In [ ]:
# the add index parameter allows you to store the index arrays as columns
gb2 = final_df.groupby(['quarter', 'nickname'], as_index=False).sum()
print(gb2)

In [ ]:
# this is the same as this operation, essentially
gb2 = final_df.groupby(['quarter', 'nickname']).sum().reset_index()
print(gb2)

In [ ]:
sub_df = final_df[['nickname', 'quarter', 'is_big', 'feat_1', 'feat_2']].copy()
sub_df.reset_index(drop=True, inplace=True)

In [ ]:
sub_df.head()

In [ ]:
gb3 = sub_df.groupby(['nickname'])
for grouped in gb3:
    print(grouped)
    print('\n')

In [ ]:
# access the groups from the groupby object
gb3.groups

In [ ]:
# get_group method to return the DataFrame for a group by name.
gb3.get_group('Beaver')

In [ ]:
# let's create a trivial function to apply to our groupby object
def do_something(df):
    
    if any(df['is_big'] == True):
        new_value = df['feat_1'].max() + df['feat_2'].max()
    else:
        new_value = 0
    return new_value

In [ ]:
gb3.apply(do_something)

In [ ]:
# let's modify this to use the group and associated df within the function
def do_something_new(grouped):
    
    group, df = grouped
    
    if any(df['is_big'] == True):
        new_value = df['feat_1'].max() + df['feat_2'].max()
    else:
        new_value = 0
    return {group: new_value}

In [ ]:
# storing results in a list
new_scores = [do_something_new(i) for i in gb3]
for i in new_scores:
    print(i)

In [ ]:
from collections import defaultdict

new_dict = defaultdict(str)
for i in gb3:
    new_dict.update(do_something_new(i))

In [ ]:
DataFrame.from_dict(new_dict, orient='index', columns=['new_feature'])

In [ ]:
chars = pd.read_csv('https://nces.ed.gov/ipeds/datacenter/data/HD2021.zip', 
                    compression='zip',
                    encoding="ISO-8859-1")
enr = pd.read_csv('https://nces.ed.gov/ipeds/datacenter/data/EFFY2021.zip',
                  compression='zip',
                  encoding="ISO-8859-1")

In [ ]:
enr.head()